In [159]:
# 1% emission -> daily inflow 15 tao
from pool import Pool

# emission per day
emission = 0.01
base_tao_inflow = 15
issued_tao = 10.4e6
alpha_out_emission = 7200 
p = Pool(netuid = 0, tao_in = 0.01, alpha_in = 1, alpha_out = 0)

# mining cost 
expert_groups = 4
miners_per_group = 7
num_miners = expert_groups * miners_per_group
cost_per_miner = 0.52 * 24
daily_mining_cost_USD = num_miners * cost_per_miner
tao_price = 350 

df = []
total_tao_invested = 0
total_tao_auto_sell = 0
for day in range(365): 

    tao_in_emission = 7200 * 0.5 * emission
    alpha_in_emission = tao_in_emission / p.price

    # emission
    p.inject(tao_in_emission, alpha_in_emission, alpha_out_emission)
    
    # auto sell
    issued_tao += 7200
    root_tao = issued_tao * 0.5
    alpha = p.alpha_in + p.alpha_out
    root_prop = root_tao * 0.18 / (root_tao * 0.18 + alpha)
    auto_sell_tao = p.sell(alpha_out_emission * 0.41 * root_prop)
    total_tao_auto_sell += auto_sell_tao

    # inflow 
    # tao_inflow = min(day * 10 / 365, 1) * base_tao_inflow
    tao_inflow = base_tao_inflow
    p.buy(tao_inflow)
    total_tao_invested += tao_inflow

    # miner sell 
    miner_sell_tao = p.sell(daily_mining_cost_USD / tao_price / p.price)    
    
    df.append({
        'tao_inflow': tao_inflow,
        'total_tao_invested': total_tao_invested,
        'net_tao_flow': tao_inflow - miner_sell_tao - auto_sell_tao,

        'tao_in': p.tao_in,
        'alpha_in': p.alpha_in,
        'alpha_out': p.alpha_out,
        'price': p.price,
        'day': day,
        'root_prop': root_prop,
        
        'miner_sell_tao': miner_sell_tao,
        'auto_sell_tao': auto_sell_tao, 
        'total_tao_auto_sell': total_tao_auto_sell
    })

df = pd.DataFrame(df)

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import math

cols = df.columns
rows = math.ceil(len(cols) / 3)

fig = make_subplots(
    rows=rows,
    cols=3,
    shared_xaxes=True,
    subplot_titles=cols,  # adds titles above each subplot
    vertical_spacing=0.03   # <<< reduce space between plots
)

for i, col in enumerate(cols):
    r = i // 3 + 1
    c = i % 3 + 1

    fig.add_trace(
        go.Scatter(x=df.index, y=df[col], mode='lines', name=col),
        row=r, col=c
    )

    # Add Y-axis label for each subplot
    fig.update_yaxes(title_text=col, row=r, col=c)

# Add one global X-axis label (cleaner)
fig.update_xaxes(title_text="Index / Time", row=rows, col=1)
fig.update_xaxes(title_text="Index / Time", row=rows, col=2)

fig.update_layout(
    height=350 * rows,
    title="Grid of Subplots with Axis Titles",
    margin=dict(
        t=40,  # top
        b=40,  # bottom
        l=50,  # left
        r=20   # right
    )
)

fig.show()
